In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [6]:
# Creating merged dataframe with genetical score
main_df_n = pd.read_csv('GWAS_clinical.csv')
score_df_n = pd.read_csv('plink_001.csv', delim_whitespace=True)
res_df_n = pd.merge(main_df_n, score_df_n, left_on='FamID', right_on='FID')

In [7]:
from sklearn.model_selection import train_test_split, StratifiedKFold
df = res_df_n.dropna()

# Genetical and non-genetical factors
df_data = df.loc[:, ['sex', 'tg', 'hdl', 'ldl', 'SCORESUM']]
df_target = df.loc[:, 'CAD']

# Only genetical factors
df_gen_data = df.loc[:, ['SCORESUM']]
df_gen_target = df.loc[:, 'CAD']

# "Leave one out" or not
leave_one_out = False
if leave_one_out:
    k = len(df)
else:
    k = 10
    
# Splitting data
kfold = StratifiedKFold(k, True, 244)
sp_data = np.array(list(kfold.split(df, df_target)))
sp_gen_data = np.array(list(kfold.split(df, df_gen_target)))

In [8]:
def kfold_train_model(model, kfold_split_indexes, df_data, df_target):
    results = []
    for train_indexes, test_indexes in kfold_split_indexes:
        x_train = df_data.iloc[train_indexes, :]
        y_train = df_target.iloc[train_indexes]
        x_test = df_data.iloc[test_indexes, :]
        y_test =  df_target.iloc[test_indexes]
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        results.append(accuracy_score(y_test, predictions))
    return np.array(results)

In [9]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs')
res = kfold_train_model(log_reg, sp_data, df_data, df_target)
res_gen = kfold_train_model(log_reg, sp_gen_data, df_gen_data, df_gen_target)
print(res.mean(), res.std())
print(res_gen.mean(), res_gen.std())

0.7533801234511384 0.03522601899041005
0.7329443058353171 0.024334465150684957


In [11]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
res = kfold_train_model(rf, sp_data, df_data, df_target)
res_gen = kfold_train_model(rf, sp_gen_data, df_gen_data, df_gen_target)
print(res.mean(), res.std())
print(res_gen.mean(), res_gen.std())

0.7471356074131723 0.02354989941157972
0.670501625160227 0.0360898506931481


In [12]:
# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100)
res = kfold_train_model(gb, sp_data, df_data, df_target)
res_gen = kfold_train_model(gb, sp_gen_data, df_gen_data, df_gen_target)
print(res.mean(), res.std())
print(res_gen.mean(), res_gen.std())

0.7581044375267046 0.02931638177792102
0.721255455350058 0.024943828169692817


In [13]:
# Naive bayess
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
res = kfold_train_model(gnb, sp_data, df_data, df_target)
res_gen = kfold_train_model(gb, sp_gen_data, df_gen_data, df_gen_target)
print(res.mean(), res.std())
print(res_gen.mean(), res_gen.std())

0.7619681987425991 0.032143204995460674
0.721255455350058 0.024943828169692817
